In [8]:
import json

In [174]:
import folium
import json
# function the visualizes the solution actions and costs on the map
# get the path and dict that contaoins roads coordinates lists
def find_indices(list_to_check, item_to_find):
    return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]
def visualize_path(path=tuple):
  with open('/content/roads.json') as f:   road_visual = json.load(f)
  center = [29.941725,31.067126] ## the map center so it starts on top of Zewail City
  m = folium.Map(location=center, zoom_start=16)  # request a map
  N_path= [] #empty array that will be filled with the roads that will be displayed on the map
  for node in path: # loop over the actions list
    #print(node)
    if node not in ['Pick','Drop']:
       node = node.split(":")[1] # split in ":" and get the seconded elelment as it is equal to the state name
    if node in road_visual: # chek if the state has coordinates and is in the dictionary
      N_path.append(node) # append  the states that will be displayed on the map
  
  actions = set(N_path)
  indc = []
  dct = {}
  for idx ,value in enumerate(actions):
      dct[value] = find_indices(N_path,value)
      
  for key in dct.keys():  # loop over the states that will be displayed on the map
      jsn = road_visual[key]  # get the json object if the state 
      dc = jsn.get("features")[0].get("geometry").get("coordinates") # get the Coordinates list
      if len(dc) == 1: #check if the coordinate list has single point so it is not a road and it will be  displayed as a marker
        dc[1], dc[0] = dc[0] , dc[1] # swap the points as folium is taking the values inverted
        #folium.Marker(dc[0], tooltip="cross point", icon=folium.Icon(color="green", icon="flag")).add_to(m) # draw a marker on the map
      else:
        for i in range(len(dc)): # loop over the Coordinates list
          dc[i][1], dc[i][0] = dc[i][0] , dc[i][1]  # swap the points as folium is taking the values inverted
        folium.Marker(dc[-1], tooltip=f"{dct[key]}",icon=folium.Icon(color="red", icon="flag")).add_to(m)
        # draw the road on the lap as a line
      
  return m  # return the map wath the path 

In [176]:
class Environment:
    '''
    Abstract base class for an (interactive) environment formulation.
    It declares the expected methods to be used to solve it.
    All the methods declared are just placeholders that throw errors if not overriden by child "concrete" classes!
    '''
    
    def __init__(self):
        '''Constructor that initializes the problem. Typically used to setup the initial state.'''
        self.state = None
    
    def actions(self):
        '''Returns an iterable with the applicable actions to the current environment state.'''
        raise NotImplementedError
    
    def apply(self, action):
        '''Applies the action to the current state of the environment and returns the new state from applying the given action to the current environment state; not necessarily deterministic.'''
        raise NotImplementedError
    
    @classmethod
    def new_random_instance(cls):
        '''Factory method to a problem instance with a random initial state.'''
        raise NotImplementedError


In [211]:
def q_learning(env, q={}, n={}, f=lambda q, n: (q+1)/(n+1),alpha=0.5, verbose=False):
    '''Q-learning implementation that trains on an environment till no more actions can be taken'''
    count = 0
    lst_actions = []
    while env.state is not None:
        count +=1
        state = env.state
        action = max(env.actions(),
                     key=lambda next_action: f(q.get((state, next_action), 0), n.get((state, next_action), 0)))
        print("  ",action, count)
        n[(state, action)] = n.get((state, action), 0) + 1
        reward = env.apply(action)
        lst_actions.append(action)
        q[(state, action)] = q.get((state, action), 0) \
                           + alpha * (reward\
                              + env.discount * max((q.get((env.state, next_action), 0) for next_action in env.actions()), default=0)
                              - q.get((state, action), 0))
   
        if count == 4000:  return q, n, lst_actions
    return q, n,lst_actions

In [195]:
from math import inf
from time import time
from itertools import count


def simulate(env_ctor, n_iterations=inf, duration=inf, **q_learning_params):
    '''A helper function to train for a fixed number of iterations or fixed time'''
    for param in ('q', 'n'): q_learning_params[param] = q_learning_params.get(param, {})
    start_time = time()
    i = count()
    while time() < start_time + duration and next(i) < n_iterations:
        env = env_ctor()
        q, n, actions = q_learning(env, **q_learning_params)

        if time() == start_time + 0.5 *duration:  print("Q and N values in the middle of learning: ",q, n, "\n")
   
    return q_learning_params['q'], q_learning_params['n'], actions

In [196]:
class Can_Collector(Environment):
    
    def __init__(self,HB_basket,NB_basket,OSB_basket,AB_basket, init_state):
        self.state =init_state ; self.HB_basket = HB_basket ; self.NB_basket = NB_basket  
        self.OSB_basket = OSB_basket ; self.AB_basket = AB_basket ; self.can_in_hand = False
        self.discount = 0.3

        with open('/content/Nodes.json') as f:   self.Nodes=  json.load(f)
        with open('/content/Actions.json') as f: self.Actions=  json.load(f)
        # dict contains all possible routes with their cost and estimated number of pumps...

        self.Road_cost= {
            'R1' : (100,0) , 'R2' : (100,0)  ,'R3' : (100,0)  , 'R4' : (100,0)   ,'R5' : (70,0)   ,'R6' : (90,0)   , 'R7' : (100,0)  ,'R8' : (100,0)   ,
                          'R9' : (100,0)  , 'R10' : (100,0)  , 'R11' : (100,0)  , 'R12' : (100,0)  , 'R13' : (110,0)  , 'R14' : (100,0)  , 'R15' : (100,0)  ,
                          'R16' : (110,0)  , 'R17' : (100,0)  , 'R18' : (110,0)  , 'R19' : (130,0)  , 'R20' : (130,0)   , 'R21' : (100,0)   , 'R22' : (110,0)   , 
                          'R23' : (80,0)  , 'R24' : (90,0)  , 'R25' : (100,0)   , 'R26' : (150,1)  , 'R27' :  (100,2) , 'R28' :  (130,3)  , 'R29' : (130,3)   ,
                          'R30' : (120,2)   , 'R31' : (150,2)  , 'R32' : (100,1)   , 'R33' : (130,2) , 'R34' : (120,1)  , 'R35' : (100,2)  , 'R36' : (100,2)  ,'R37':(100,0),
                          'Academic':(80,0),'Helmy':(80,0),'Nano':(80,0),'mangment':(80,0),'One stop':(80,0),'service':(80,0),'Dorms':(80,0),'parking':(80,0),'masged':(80,0),
                          'Gate1':(100,0),'Gate2':(100,0),'Gate3':(100,0),'Gate4':(100,0),'Gate5':(100,0),
        }

    def actions(self):
        if self.state==None : available_actions = []
        else: available_actions = self.Actions[self.state].copy()
        
        if self.state in ["Nano", "Helmy"]:
            if self.can_in_hand: available_actions.remove("Pick") 
        if self.state in ["One stop", "Academic"]:
            if not self.can_in_hand: available_actions.remove("Drop")
            
        if self.state == "Nano" and self.NB_basket == 0 and "Pick" in available_actions: available_actions.remove("Pick")
        elif self.state == "Helmy" and self.HB_basket == 0 and "Pick" in available_actions: available_actions.remove("Pick")
        elif self.state == "One stop" and self.OSB_basket == 4 and "Drop" in available_actions: available_actions.remove("Drop")
        elif self.state == "Academic" and self.AB_basket == 4 and "Drop" in available_actions: available_actions.remove("Drop")



        return (available_actions)

    
    def apply(self, action):
        print("Buskets:",'\n',"Nano: ", self.NB_basket, ", Helmy: ",self.HB_basket, ", Academic: ",self.AB_basket, ", One stop: ",self.OSB_basket)
        if action == "Pick":
            if self.state == "Nano" and self.NB_basket != 0: 
                self.can_in_hand = True ; self.NB_basket -=1 ; return +1000

            elif self.state == "Helmy" and self.HB_basket != 0: 
                self.can_in_hand = True ; self.HB_basket -=1 ; return +1000
            
            elif self.state == "One stop" :#self.can_in_hand = True ; self.OSB_basket -=1; 
                return -1000
            elif self.state == "Academic" :#self.can_in_hand = True ; self.AB_basket -=1; 
                return -1000


        elif action == "Drop":
            if self.state == "One stop" and self.OSB_basket != 4: 
                self.can_in_hand = False ; self.OSB_basket +=1 ; 
                if self.OSB_basket == 4 and self.AB_basket == 4: self.state = None
                return +1000
             
            elif self.state == "Academic" and self.AB_basket != 4: 
                self.can_in_hand = False ; self.AB_basket +=1  ; 
                if self.OSB_basket == 4 and self.AB_basket == 4: self.state = None
                return +1000
           
            elif self.state == "Nano" :#self.can_in_hand = False ; self.NB_basket +=1;  
              return -1000
            elif self.state == "Helmy" :#self.can_in_hand = False ; self.HB_basket +=1;  
              return -1000

        else:
            action = action.split(":")[1]
            self.state =action
            return  -(self.Road_cost[action][0] + 20* self.Road_cost[action][1])
    @classmethod
    def new_instance(cls,HB_basket,NB_basket,OSB_basket,AB_basket, init_state):
        return cls(HB_basket,NB_basket,OSB_basket,AB_basket, init_state)

In [207]:
from random import random
q,n = {},{}
simulate( lambda: Can_Collector.new_instance(HB_basket=4,OSB_basket=0,NB_basket=4,AB_basket=0, init_state= 'Gate1'),duration=60 , q=q, n=n,  f=lambda q, n: random())
print("------------------------------------------------------------------------------------------: \n")


Streaming output truncated to the last 5000 lines.
   Move to:R6 2335
Buskets: 
 Nano:  0 , Helmy:  4 , Academic:  2 , One stop:  2
   Move to:R11 2336
Buskets: 
 Nano:  0 , Helmy:  4 , Academic:  2 , One stop:  2
   Move to:R10 2337
Buskets: 
 Nano:  0 , Helmy:  4 , Academic:  2 , One stop:  2
   Move to:R25 2338
Buskets: 
 Nano:  0 , Helmy:  4 , Academic:  2 , One stop:  2
   Move to:R10 2339
Buskets: 
 Nano:  0 , Helmy:  4 , Academic:  2 , One stop:  2
   Move to:R11 2340
Buskets: 
 Nano:  0 , Helmy:  4 , Academic:  2 , One stop:  2
   Move to:R7 2341
Buskets: 
 Nano:  0 , Helmy:  4 , Academic:  2 , One stop:  2
   Move to:R37 2342
Buskets: 
 Nano:  0 , Helmy:  4 , Academic:  2 , One stop:  2
   Move to:R7 2343
Buskets: 
 Nano:  0 , Helmy:  4 , Academic:  2 , One stop:  2
   Move to:R6 2344
Buskets: 
 Nano:  0 , Helmy:  4 , Academic:  2 , One stop:  2
   Move to:R4 2345
Buskets: 
 Nano:  0 , Helmy:  4 , Academic:  2 , One stop:  2
   Move to:R6 2346
Buskets: 
 Nano:  0 , Helmy:  4 ,

## **Q values in the middle of training: **
{('Gate1', 'Move to:R13'): -151.91351185877903,
 ('R13', 'Move to:Gate1'): -141.90484855597276,
 ('R13', 'Move to:R14'): -140.3569895363167,
 ('R14', 'Move to:R15'): -134.5003903537463,
 ('R15', 'Move to:parking'): -114.89977965943237,
 ('parking', 'Move to:R16'): -144.03961537081335,
 ('R16', 'Move to:R2'): -127.76570688485018,
 ('R2', 'Move to:R17'): -129.59957538332623,
 ('R17', 'Move to:R1'): -126.95232959025446,
 ('R1', 'Move to:R36'): -172.0828033206418,
 ('R36', 'Move to:R35'): -173.8854750889583,
 ('R35', 'Move to:R33'): -214.83516426374604,
 ('R33', 'Move to:R32'): -164.83516465824277,
 ('R32', 'Move to:R31'): -232.1978021977551,
 ('R31', 'Move to:Dorms'): -140.6593406593342,
 ('Dorms', 'Move to:R30'): -202.19780219780017,
 ('R30', 'Move to:Dorms'): -140.6593406593394,
 ('Dorms', 'Move to:R29'): -231.97235104322323,
 ('R29', 'Move to:Dorms'): -140.65934065933777,
 ('R29', 'Move to:R30'): -202.1978021977996,
 ('R30', 'Move to:R29'): -232.1330119629162,
 ('R29', 'Move to:R28'): -181.5104834779749,
 ('R28', 'Move to:R27'): -133.87797425843183,
 ('R27', 'Move to:R26'): -210.6146923325615,
 ('R26', 'Move to:R12'): -138.57511378756845,
 ('R12', 'Move to:R10'): -133.88201994284046,
 ('R10', 'Move to:R9'): -131.45390586741115,
 ('R9', 'Move to:R25'): -116.61629158649268,
 ('R25', 'Move to:R10'): -133.37806976369586,
 ('R10', 'Move to:R12'): -139.73371434784065,
 ('R12', 'Move to:R13'): -151.93677112790982,
 ('R14', 'Move to:R13'): -151.9409751917275,
 ('Gate1', 'Move to:R12'): -139.76908635624477,
 ('R12', 'Move to:Gate1'): -141.7836001999815,
 ('R12', 'Move to:R11'): -132.3574815815229,
 ('R11', 'Move to:R6'): -101.0683886726409,
 ('R6', 'Move to:R7'): -130.11126998169533,
 ('R7', 'Move to:mangment'): -118.51652769833902,
 ('mangment', 'Move to:R7'): -130.29580998205643,
 ('R7', 'Move to:R6'): -100.04171025257048,
 ('R6', 'Move to:R4'): -120.86493966395312,
 ('R4', 'Move to:R2'): -127.7823200538605,
 ('R2', 'Move to:parking'): -115.20481479867308,
 ('parking', 'Move to:R15'): -134.5488307505037,
 ('R16', 'Move to:R17'): -132.12100465858788,
 ('R17', 'Move to:R34'): -171.824249558754,
 ('R34', 'Move to:R35'): -174.53019281609818,
 ('R33', 'Move to:R35'): -173.05487981021275,
 ('R35', 'Move to:service'): -118.41305597540499,
 ('service', 'Move to:R34'): -174.85087874577414,
 ('R34', 'Move to:R17'): -131.91999034268648,
 ('R17', 'Move to:R19'): -161.94734887199814,
 ('R19', 'Move to:R18'): -134.47860977301497,
 ('R18', 'Move to:service'): -116.67502787330723,
 ('service', 'Move to:R19'): -162.64317909088976,
 ('R19', 'Move to:R34'): -171.80467567375547,
 ('R34', 'Move to:R19'): -163.4036758754674,
 ('R19', 'Move to:service'): -115.74493054183253,
 ('service', 'Move to:R35'): -174.81396237311532,
 ('service', 'Move to:R18'): -130.9251892261855,
 ('R18', 'Move to:R3'): -128.80805339609373,
 ('R3', 'Move to:R2'): -127.77277185321799,
 ('R2', 'Move to:R4'): -118.89006763590957,
 ('R4', 'Move to:R5'): -32.92230096752188,
 ('R5', 'Move to:Helmy'): 182.63114198905907,
 ('Helmy', 'Move to:R5'): -26.390941464145754,
 ('R5', 'Move to:R24'): -96.8501534878622,
 ('R24', 'Move to:Academic'): -94.09855394625427,
 ('Academic', 'Move to:R3'): -127.85685168188816,
 ('R4', 'Move to:R3'): -128.5323754808331,
 ('R3', 'Move to:R18'): -134.33213280307928,
 ('R18', 'Move to:R19'): -160.82833948111139,
 ('R19', 'Move to:R17'): -132.28352448880221,
 ('R17', 'Move to:R36'): -173.19248256237995,
 ('R36', 'Move to:service'): -117.51065422177831,
 ('R19', 'Move to:R20'): -166.13817903999205,
 ('R20', 'Move to:R32'): -164.83514970694316,
 ('R31', 'Move to:Gate4'): -149.45054174694764,
 ('Gate4', 'Move to:R32'): -164.8351643771975,
 ('R31', 'Move to:R32'): -164.83516420168905,
 ('R32', 'Move to:R20'): -166.13821977802894,
 ('R20', 'Move to:R19'): -160.72071732228287,
 ('R17', 'Move to:R2'): -133.04413826154547,
 ('R15', 'Move to:R16'): -144.0515789592937,
 ('R16', 'Move to:R1'): -126.66145462039056,
 ('R1', 'Move to:Gate5'): -137.65161434148635,
 ('Gate5', 'Move to:R1'): -126.63120954157534,
 ('R1', 'Move to:R16'): -144.41556907368832,
 ('R16', 'Move to:parking'): -115.01600303401797,
 ('parking', 'Move to:R2'): -130.57533492432862,
 ('R4', 'Move to:R6'): -98.40269654027114,
 ('R6', 'Move to:R5'): -17.984326693327827,
 ('R24', 'Move to:R5'): -17.177551883162046,
 ('R5', 'Move to:R4'): -118.88347623546888,
 ('R3', 'Move to:R1'): -126.87396290128615,
 ('R1', 'Move to:Academic'): -92.69180929787436,
 ('Academic', 'Move to:R37'): -113.57611070860159,
 ('R37', 'Move to:Academic'): -101.11053331226256,
 ('Academic', 'Move to:R18'): -127.4782909865698,
 ('R18', 'Move to:R17'): -132.2267375175991,
 ('R17', 'Move to:R16'): -143.6705527993179,
 ('R16', 'Move to:R36'): -172.77800560891234,
 ('R36', 'Move to:R16'): -144.26074099934397,
 ('R1', 'Move to:R2'): -130.57163070502048,
 ('R24', 'Move to:Nano'): -98.13228763612338,
 ('Nano', 'Move to:R23'): -74.83424984314149,
 ('R23', 'Move to:Nano'): -97.43416102170343,
 ('Nano', 'Drop'): -1017.4124124543741,
 ('Nano', 'Move to:R24'): -97.79755270473271,
 ('R2', 'Move to:R3'): -120.62309945385634,
 ('R3', 'Move to:Academic'): -99.94121445070353,
 ('Academic', 'Move to:R24'): -95.9008470143421,
 ('R24', 'Move to:R23'): -65.81517072048632,
 ('R23', 'Move to:R22'): -120.884051063247,
 ('R22', 'Move to:One stop'): -82.40231349040846,
 ('One stop', 'Move to:R23'): -80.30511418552803,
 ('R23', 'Move to:R24'): -95.85648657324258,
 ('R5', 'Move to:Academic'): -95.38860541951439,
 ('Academic', 'Move to:R1'): -126.62186756491579,
 ('R34', 'Move to:R33'): -214.8351642705373,
 ('R33', 'Move to:R31'): -232.19780219759383,
 ('R30', 'Move to:R31'): -232.1978021977129,
 ('Dorms', 'Move to:R31'): -232.19780219779167,
 ('R20', 'Move to:masged'): -120.05229331201276,
 ('masged', 'Move to:R21'): -132.59990374277461,
 ('R21', 'Move to:masged'): -119.84027784055813,
 ('masged', 'Move to:R20'): -166.0563697245144,
 ('R19', 'Move to:R32'): -164.83515862445182,
 ('R32', 'Move to:R19'): -162.73243840148257,
 ('R17', 'Move to:R18'): -130.37321354514904,
 ('R17', 'Move to:service'): -114.93021499793286,
 ('service', 'Move to:R36'): -173.3770426364568,
 ('R36', 'Move to:R17'): -128.71282916189227,
 ('service', 'Move to:R17'): -132.8815839716984,
 ('R21', 'Move to:R28'): -165.88322627761485,
 ('R28', 'Move to:R29'): -231.75767310118027,
 ('R29', 'Move to:Gate3'): -154.19880431224453,
 ('Gate3', 'Move to:R28'): -190.7240673384553,
 ('R28', 'Move to:Gate2'): -138.1646238346044,
 ('Gate2', 'Move to:R27'): -113.66799995484432,
 ('R27', 'Move to:Gate2'): -146.9269244691767,
 ('Gate2', 'Move to:R28'): -198.80607450516203,
 ('R27', 'Move to:One stop'): -37.82467302155099,
 ('One stop', 'Move to:R22'): -120.85897068027985,
 ('R22', 'Move to:R28'): -183.10476021790402,
 ('R28', 'Move to:Gate3'): -152.28923222193748,
 ('Gate3', 'Move to:R29'): -232.1654739811716,
 ('R27', 'Move to:R28'): -170.23079659386096,
 ('R28', 'Move to:R22'): -126.18022521987956,
 ('R22', 'Move to:R21'): -135.83661439544565,
 ('R21', 'Move to:R29'): -232.1055682469854,
 ('R22', 'Move to:R23'): -71.25997075593986,
 ('R23', 'Move to:One stop'): -24.618206362342463,
 ('R21', 'Move to:R20'): -166.13786175704882,
 ('R35', 'Move to:R34'): -171.72132007493053,
 ('R34', 'Move to:R18'): -122.1103123618222,
 ('R18', 'Move to:R1'): -127.17176435552778,
 ('Academic', 'Move to:R8'): -128.45500279399,
 ('R8', 'Move to:R7'): -130.20116982845855,
 ('R7', 'Move to:R37'): -113.1462951077657,
 ('R37', 'Move to:R24'): -45.060229280570795,
 ('R7', 'Move to:R11'): -131.24420585238477,
 ('R11', 'Move to:R7'): -130.10598729599414,
 ('R7', 'Move to:R8'): -128.31578876493148,
 ('R11', 'Move to:R12'): -139.81034995440982,
 ('Gate1', 'Move to:R26'): -211.2332079587218,
 ('R26', 'Move to:R13'): -151.8987622839851,
 ('R13', 'Move to:R12'): -139.82264731793398,
 ('R18', 'Move to:Academic'): -91.8211463919879,
 ('R1', 'Move to:R17'): -132.26127224816958,
 ('R2', 'Move to:R16'): -139.0503025005607,
 ('R1', 'Move to:R3'): -128.64004441947696,
 ('Academic', 'Pick'): -1017.5447347048218,
 ('R6', 'Move to:R11'): -132.2038061119539,
 ('R11', 'Move to:mangment'): -118.3875153166922,
 ('mangment', 'Move to:R8'): -128.58282989714712,
 ('R8', 'Move to:mangment'): -118.58436570453017,
 ('mangment', 'Move to:R11'): -132.7967966625752,
 ('R11', 'Move to:R10'): -133.4289833452419,
 ('R10', 'Move to:mangment'): -118.65839014858864,
 ('R3', 'Move to:R4'): -113.38461407839785,
 ('R35', 'Move to:R36'): -174.05594517222045,
 ('R15', 'Move to:R14'): -140.3572276974978,
 ('R16', 'Move to:Gate5'): -137.6128826025958,
 ('Gate5', 'Move to:R16'): -144.0639801269723,
 ('R16', 'Move to:R15'): -134.4870661553776,
 ('R1', 'Move to:R18'): -136.7403240222344,
 ('R18', 'Move to:R34'): -173.85038133224162,
 ('R32', 'Move to:R33'): -214.83516431208375,
 ('R33', 'Move to:Gate4'): -149.45054914236093,
 ('Gate4', 'Move to:R33'): -214.83516348757468,
 ('R33', 'Move to:R34'): -171.62938970124168,
 ('Gate4', 'Move to:R31'): -232.19780219744646,
 ('R20', 'Move to:R21'): -135.7630359283549,
 ('R21', 'Move to:R22'): -123.98122340264382,
 ('R22', 'Move to:R29'): -231.84773006307822,
 ('R22', 'Move to:R25'): -111.70072546991341,
 ('R25', 'Move to:One stop'): -97.99065680150215,
 ('One stop', 'Move to:R27'): -136.84871829783242,
 ('R25', 'Move to:R9'): -132.9950365761947,
 ('R9', 'Move to:R8'): -128.12071689502,
 ('R4', 'Move to:parking'): -115.33428996817739,
 ('parking', 'Move to:R4'): -116.07829601995806,
 ('R36', 'Move to:Gate5'): -137.5543988560769,
 ('Gate5', 'Move to:R36'): -174.50415005264452,
 ('R2', 'Move to:R36'): -173.98852774492883,
 ('R36', 'Move to:R2'): -127.99144708581596,
 ('R2', 'Move to:R1'): -127.17528639737431,
 ('R36', 'Move to:R1'): -127.43532353830905,
 ('R10', 'Move to:R25'): -116.41812221238791,
 ('R23', 'Move to:R25'): -115.50266346229864,
 ('One stop', 'Move to:R25'): -116.28618144235341,
 ('R25', 'Move to:R22'): -123.2462260569106,
 ('R28', 'Move to:One stop'): -12.939439129784319,
 ('R5', 'Move to:R6'): -96.14005488639546,
 ('R8', 'Move to:Academic'): -94.01130467705744,
 ('Academic', 'Move to:R5'): -45.370853024370476,
 ('R31', 'Move to:R30'): -202.19780219777263,
 ('R28', 'Move to:R21'): -135.58038175131315,
 ('R25', 'Move to:R23'): -72.2142963944711,
 ('Nano', 'Pick'): 985.9043911144422,
 ('Academic', 'Drop'): 984.0944827321129,
 ('R37', 'Move to:R7'): -131.7125769126037,
 ('R9', 'Move to:mangment'): -118.46925131343735,
 ('R8', 'Move to:R9'): -134.94096096376398,
 ('mangment', 'Move to:R10'): -134.0082879400672,
 ('R10', 'Move to:R11'): -132.2166008488781,
 ('R9', 'Move to:R10'): -133.72415954093447,
 ('mangment', 'Move to:R9'): -134.76292152242308,
 ('Helmy', 'Drop'): -995.8414892449332,
 ('Helmy', 'Pick'): 977.042562393156,
 ('One stop', 'Pick'): -1010.1189702268881,
 ('R26', 'Move to:Gate1'): -141.31219268705792,
 ('R31', 'Move to:R33'): -214.83516440216917,
 ('R32', 'Move to:Gate4'): -149.45054903876664,
 ('R34', 'Move to:service'): -118.29733717221126,
 ('One stop', 'Move to:R28'): -188.75336254884047,
 ('R26', 'Move to:R27'): -150.81390406559635,
 ('One stop', 'Drop'): 982.4725622945095}

Q={('Gate1', 'Move to:R26'): -211.4471110543528, 
('R26', 'Move to:R13'): -151.5178977339625, 
('R13', 'Move to:R12'): -138.94621961742286, 
('R12', 'Move to:R10'): -129.21303303573592, 
('R10', 'Move to:R12'): -138.77061934567976, 
('R12', 'Move to:R13'): -151.60147817926355, 
('R13', 'Move to:R14'): -140.05724427988406, 
('R14', 'Move to:R15'): -133.48154616650248, 
('R15', 'Move to:R14'): -140.05211192928232, 
('R15', 'Move to:parking'): -111.47293108477314, 
('parking', 'Move to:R16'): -143.8824366399627, 
('R16', 'Move to:parking'): -115.17665773493553, 
('parking', 'Move to:R15'): -133.47322515798172, 
('R14', 'Move to:R13'): -151.61252096788945, 
('R13', 'Move to:Gate1'): -141.5530924139997, 
('R26', 'Move to:Gate1'): -141.53714335840942, 
('Gate1', 'Move to:R12'): -138.67160368367416, 
('R10', 'Move to:R9'): -130.2813640174753, 
('R9', 'Move to:mangment'): -112.96949809977156, 
('mangment', 'Move to:R8'): -120.62064158404827, 
('R8', 'Move to:R7'): -130.5139191458606, 
('R7', 'Move to:R11'): -132.0815706478167, 
('R11', 'Move to:R10'): -128.80536876391693, 
('R9', 'Move to:R25'): -103.96140349380411, 
('R25', 'Move to:One stop'): -90.2666581401051, 
('One stop', 'Move to:R27'): -143.95506798093555, 
('R27', 'Move to:Gate2'): -150.07403584895656, 
('Gate2', 'Move to:R27'): -166.90891109468566, 
('R27', 'Move to:R28'): -165.13025590741194, 
('R28', 'Move to:One stop'): 197.51793554961654, 
('One stop', 'Pick'): -897.5504754508681, 
('One stop', 'Move to:R23'): -108.98723779125726, 
('R23', 'Move to:R24'): -111.64855972411968, 
('R24', 'Move to:R5'): -99.5460374588873, 
('R5', 'Move to:Helmy'): -107.69276928098007, 
('Helmy', 'Drop'): -1017.6660507140951, 
('Helmy', 'Pick'): 971.7036210448056, 
('Helmy', 'Move to:R5'): -114.88589070453506, 
('R5', 'Move to:R4'): -116.07197378126749, 
('R4', 'Move to:R2'): -130.06344411320367, 
('R2', 'Move to:R16'): -142.69782362152318, ('R16', 'Move to:R1'): -120.22512837453274, ('R1', 'Move to:R16'): -144.17906701644458, ('R16', 'Move to:R36'): -173.93538124391682, ('R36', 'Move to:R2'): -126.10029523786866, ('R2', 'Move to:R3'): -104.65758400098468, ('R3', 'Move to:Academic'): -74.45033615959939, ('Academic', 'Move to:R24'): -101.15283551108575, ('R24', 'Move to:Nano'): -98.37053587298793, ('Nano', 'Move to:R23'): -106.2239003306852, ('R23', 'Move to:R25'): -111.5935599389924, ('R25', 'Move to:R23'): -86.56674421235856, ('Nano', 'Drop'): -863.2305817911501, ('Nano', 'Move to:R24'): -110.68307084077453, ('R5', 'Move to:R6'): -113.59026120585088, ('R6', 'Move to:R5'): -80.78371436017065, ('R4', 'Move to:R5'): -77.53603680197423, ('R5', 'Move to:Academic'): 52.17867578657197, ('R24', 'Move to:R23'): -102.38609337461475, ('R23', 'Move to:Nano'): -107.92029868954164, ('R23', 'Move to:R22'): -111.05935704132634, ('R22', 'Move to:R28'): -182.3824005672672, ('One stop', 'Move to:R25'): -108.67765296312653, ('R25', 'Move to:R22'): -100.2751550999541, ('R22', 'Move to:R29'): -232.19780219780216, ('R29', 'Move to:Dorms'): -140.65934065934061, ('Dorms', 'Move to:R29'): -232.19780219780216,

In [210]:
Q={('Gate1', 'Move to:R26'): -211.4471110543528, ('R26', 'Move to:R13'): -151.5178977339625, ('R13', 'Move to:R12'): -138.94621961742286, ('R12', 'Move to:R10'): -129.21303303573592, ('R10', 'Move to:R12'): -138.77061934567976, ('R12', 'Move to:R13'): -151.60147817926355, ('R13', 'Move to:R14'): -140.05724427988406, ('R14', 'Move to:R15'): -133.48154616650248, ('R15', 'Move to:R14'): -140.05211192928232, ('R15', 'Move to:parking'): -111.47293108477314, ('parking', 'Move to:R16'): -143.8824366399627, ('R16', 'Move to:parking'): -115.17665773493553, ('parking', 'Move to:R15'): -133.47322515798172, ('R14', 'Move to:R13'): -151.61252096788945, ('R13', 'Move to:Gate1'): -141.5530924139997, ('R26', 'Move to:Gate1'): -141.53714335840942, ('Gate1', 'Move to:R12'): -138.67160368367416, ('R10', 'Move to:R9'): -130.2813640174753, ('R9', 'Move to:mangment'): -112.96949809977156, ('mangment', 'Move to:R8'): -120.62064158404827, ('R8', 'Move to:R7'): -130.5139191458606, ('R7', 'Move to:R11'): -132.0815706478167, ('R11', 'Move to:R10'): -128.80536876391693, ('R9', 'Move to:R25'): -103.96140349380411, ('R25', 'Move to:One stop'): -90.2666581401051, ('One stop', 'Move to:R27'): -143.95506798093555, ('R27', 'Move to:Gate2'): -150.07403584895656, ('Gate2', 'Move to:R27'): -166.90891109468566, ('R27', 'Move to:R28'): -165.13025590741194, ('R28', 'Move to:One stop'): 197.51793554961654, ('One stop', 'Pick'): -897.5504754508681, ('One stop', 'Move to:R23'): -108.98723779125726, ('R23', 'Move to:R24'): -111.64855972411968, ('R24', 'Move to:R5'): -99.5460374588873, ('R5', 'Move to:Helmy'): -107.69276928098007, ('Helmy', 'Drop'): -1017.6660507140951, ('Helmy', 'Pick'): 971.7036210448056, ('Helmy', 'Move to:R5'): -114.88589070453506, ('R5', 'Move to:R4'): -116.07197378126749, ('R4', 'Move to:R2'): -130.06344411320367, ('R2', 'Move to:R16'): -142.69782362152318, ('R16', 'Move to:R1'): -120.22512837453274, ('R1', 'Move to:R16'): -144.17906701644458, ('R16', 'Move to:R36'): -173.93538124391682, ('R36', 'Move to:R2'): -126.10029523786866, ('R2', 'Move to:R3'): -104.65758400098468, ('R3', 'Move to:Academic'): -74.45033615959939, ('Academic', 'Move to:R24'): -101.15283551108575, ('R24', 'Move to:Nano'): -98.37053587298793, ('Nano', 'Move to:R23'): -106.2239003306852, ('R23', 'Move to:R25'): -111.5935599389924, ('R25', 'Move to:R23'): -86.56674421235856, ('Nano', 'Drop'): -863.2305817911501, ('Nano', 'Move to:R24'): -110.68307084077453, ('R5', 'Move to:R6'): -113.59026120585088, ('R6', 'Move to:R5'): -80.78371436017065, ('R4', 'Move to:R5'): -77.53603680197423, ('R5', 'Move to:Academic'): 52.17867578657197, ('R24', 'Move to:R23'): -102.38609337461475, ('R23', 'Move to:Nano'): -107.92029868954164, ('R23', 'Move to:R22'): -111.05935704132634, ('R22', 'Move to:R28'): -182.3824005672672, ('One stop', 'Move to:R25'): -108.67765296312653, ('R25', 'Move to:R22'): -100.2751550999541, ('R22', 'Move to:R29'): -232.19780219780216, ('R29', 'Move to:Dorms'): -140.65934065934061, ('Dorms', 'Move to:R29'): -232.19780219780216, ('Dorms', 'Move to:R31'): -232.19780219780216, ('R31', 'Move to:Dorms'): -140.65934065934061, ('R29', 'Move to:R28'): -145.81357888252506, ('R28', 'Move to:R29'): -232.19780219780216, ('R29', 'Move to:Gate3'): -157.25516578666384, ('Gate3', 'Move to:R28'): -159.40106744515134, ('R28', 'Move to:Gate2'): -150.18556271686504, ('Gate2', 'Move to:R28'): -162.2872802205004, ('R28', 'Move to:R22'): -109.76397181571099, ('R22', 'Move to:R21'): -124.91005590390648, ('R21', 'Move to:R29'): -232.19780219780216, ('R29', 'Move to:R30'): -202.19780219780216, ('R30', 'Move to:R31'): -232.19780219780216, ('R31', 'Move to:R30'): -202.19780219780216, ('R31', 'Move to:R33'): -214.8351648351648, ('R33', 'Move to:R34'): -168.30706791622626, ('R34', 'Move to:R17'): -134.70707902507854, ('R17', 'Move to:R18'): -133.99525403002912, ('R18', 'Move to:R17'): -130.44168350648198, ('R17', 'Move to:R1'): -120.02985672586553, ('R1', 'Move to:R36'): -173.4963962530644, ('R2', 'Move to:R4'): -119.9456104913053, ('Academic', 'Move to:R5'): -100.55644750743045, ('R6', 'Move to:R11'): -132.025381836029, ('R11', 'Move to:mangment'): -115.06983659527583, ('R8', 'Move to:R9'): -129.91637929563348, ('R9', 'Move to:R8'): -118.84795735653029, ('R8', 'Move to:Academic'): -83.9247160994122, ('Academic', 'Move to:R37'): -100.61526465470541, ('R37', 'Move to:Academic'): -66.71133728091606, ('Academic', 'Move to:R1'): -115.03971381140755, ('R36', 'Move to:R16'): -143.61065927805674, ('R1', 'Move to:R18'): -136.64359596183047, ('R18', 'Move to:R19'): -161.29835823823493, ('R19', 'Move to:R34'): -174.97358554664655, ('R34', 'Move to:service'): -119.76971995647153, ('service', 'Move to:R17'): -134.5217054038314, ('R17', 'Move to:service'): -119.00484727623603, ('service', 'Move to:R19'): -164.68235798422796, ('R19', 'Move to:R18'): -124.6382090062414, ('R18', 'Move to:R3'): -102.79169031904877, ('R3', 'Move to:R18'): -131.21224152077673, ('R18', 'Move to:Academic'): -89.73508002552266, ('Academic', 'Pick'): -1000.7370602742612, ('Academic', 'Drop'): 986.7316655903771, ('R24', 'Move to:Academic'): -100.22579875959852, ('R37', 'Move to:R24'): -75.62756649435758, ('R6', 'Move to:R4'): -119.13430181649996, ('R4', 'Move to:R3'): -120.90315119700978, ('R3', 'Move to:R4'): -121.80899584204545, ('R4', 'Move to:parking'): -114.95340848497419, ('parking', 'Move to:R2'): -128.70878189760145, ('R2', 'Move to:R36'): -172.54807934780496, ('R36', 'Move to:R1'): -123.92875323321809, ('R36', 'Move to:service'): -118.2163746633238, ('R19', 'Move to:R32'): -164.8351648351648, ('R32', 'Move to:Gate4'): -149.45054945054943, ('Gate4', 'Move to:R33'): -214.8351648351648, ('R33', 'Move to:R31'): -232.19780219780216, ('R30', 'Move to:R29'): -232.19780219780216, ('R28', 'Move to:R27'): -144.01561793986872, ('R28', 'Move to:R21'): -126.55427779259767, ('R21', 'Move to:masged'): -118.14586623522851, ('masged', 'Move to:R20'): -165.78788364359932, ('R20', 'Move to:R32'): -164.8351648351648, ('R32', 'Move to:R31'): -232.19780219780216, ('R33', 'Move to:R35'): -173.99137611394863, ('R35', 'Move to:R33'): -214.8351648351648, ('R35', 'Move to:R34'): -175.51520055354584, ('R34', 'Move to:R33'): -214.8351648351648, ('R34', 'Move to:R19'): -161.35051811704102, ('R19', 'Move to:R20'): -165.84199184663998, ('R32', 'Move to:R19'): -162.1294214708741, ('R19', 'Move to:service'): -118.1433853502651, ('service', 'Move to:R18'): -135.52690062212008, ('R18', 'Move to:service'): -117.30888210398456, ('service', 'Move to:R34'): -175.10369278413623, ('R34', 'Move to:R35'): -172.91963559314797, ('R35', 'Move to:service'): -117.50152964901663, ('service', 'Move to:R36'): -173.8533902355427, ('R16', 'Move to:Gate5'): -134.11583214648488, ('Gate5', 'Move to:R36'): -174.00936637518652, ('R16', 'Move to:R2'): -128.74785531911635, ('R2', 'Move to:R17'): -134.6445216797484, ('R17', 'Move to:R16'): -144.25808676737077, ('Gate5', 'Move to:R1'): -118.20409856201165, ('R1', 'Move to:R17'): -131.10831844720138, ('R17', 'Move to:R36'): -174.44835363041722, ('R36', 'Move to:Gate5'): -133.18137838780325, ('R16', 'Move to:R17'): -134.455061416143, ('R17', 'Move to:R34'): -173.26952564652566, ('service', 'Move to:R35'): -174.83972560108526, ('R35', 'Move to:R36'): -174.29922142823563, ('R1', 'Move to:R3'): -118.4609050983713, ('Gate4', 'Move to:R32'): -164.8351648351648, ('R33', 'Move to:Gate4'): -149.45054945054943, ('R33', 'Move to:R32'): -164.79315214943244, ('R19', 'Move to:R17'): -133.34070619593808, ('R3', 'Move to:R2'): -126.07536615590567, ('R2', 'Move to:parking'): -114.92942582434574, ('R1', 'Move to:R2'): -125.92830052872063, ('R17', 'Move to:R19'): -160.65921272308984, ('R32', 'Move to:R20'): -165.84025563607395, ('R20', 'Move to:R19'): -162.80113890692138, ('R17', 'Move to:R2'): -125.92482018010207, ('R2', 'Move to:R1'): -121.86490165771447, ('R18', 'Move to:R1'): -117.98290771235541, ('R1', 'Move to:Academic'): -89.72226996697381, ('Academic', 'Move to:R18'): -135.83225822958022, ('R1', 'Move to:Gate5'): -132.7481593319514, ('R20', 'Move to:R21'): -124.62322152320652, ('R21', 'Move to:R22'): -77.93986523125679, ('Gate3', 'Move to:R29'): -232.19780219780216, ('R28', 'Move to:Gate3'): -154.63202680050634, ('One stop', 'Move to:R28'): -170.7894877696209, ('R31', 'Move to:R32'): -164.8350007231112, ('Gate4', 'Move to:R31'): -232.19780219780216, ('R22', 'Move to:R25'): -74.324177992102, ('R36', 'Move to:R35'): -175.05530704380624, ('R16', 'Move to:R15'): -133.45661129952023, ('R10', 'Move to:R11'): -129.86981674055977, ('R11', 'Move to:R12'): -138.5723173871935, ('R12', 'Move to:R11'): -132.9232510717124, ('R8', 'Move to:mangment'): -109.88711480423241, ('Academic', 'Move to:R8'): -120.30430130564383, ('R7', 'Move to:mangment'): -104.5758309646817, ('mangment', 'Move to:R11'): -131.31177986432988, ('R11', 'Move to:R6'): -113.47201514149342, ('R6', 'Move to:R7'): -129.93469129474425, ('R7', 'Move to:R6'): -114.04929582142225, ('R4', 'Move to:R6'): -109.85144883762325, ('R15', 'Move to:R16'): -143.56738132013209, ('Nano', 'Pick'): 985.8531798765166, ('R34', 'Move to:R18'): -134.91600509394598, ('R20', 'Move to:masged'): -119.55325482992973, ('masged', 'Move to:R21'): -124.51849729902662, ('R21', 'Move to:R28'): -193.75047363271938, ('R23', 'Move to:One stop'): -108.1800261079068, ('R27', 'Move to:R26'): -211.46402617430545, ('Gate1', 'Move to:R13'): -151.56736027261184, ('Gate5', 'Move to:R16'): -143.26378412102372, ('R36', 'Move to:R17'): -133.06233559278746, ('R32', 'Move to:R33'): -214.8351648351648, ('R31', 'Move to:Gate4'): -149.45054945054943, ('R18', 'Move to:R34'): -175.22042477667395, ('Academic', 'Move to:R3'): -116.87966079276485, ('Dorms', 'Move to:R30'): -202.19780219780216, ('R25', 'Move to:R10'): -128.41988578881745, ('mangment', 'Move to:R7'): -131.0514378307137, ('R7', 'Move to:R8'): -121.08288950673241, ('R5', 'Move to:R24'): -111.92589975953868, ('One stop', 'Drop'): 983.8078724330985, ('One stop', 'Move to:R22'): -118.40227026834629, ('R22', 'Move to:One stop'): -75.67606701094024, ('R30', 'Move to:Dorms'): -140.65934065934061, ('R11', 'Move to:R7'): -130.18501346348518, ('mangment', 'Move to:R9'): -130.76831081413042, ('R12', 'Move to:Gate1'): -141.51007177900536, ('R7', 'Move to:R37'): -105.6564880979654, ('R10', 'Move to:mangment'): -112.79991154597302, ('R37', 'Move to:R7'): -124.4398683727472, ('R25', 'Move to:R9'): -129.05089466218178, ('R26', 'Move to:R27'): -167.43028712148413, ('R21', 'Move to:R20'): -165.77159828918093, ('R27', 'Move to:One stop'): -11.410394075094558, ('R3', 'Move to:R1'): -120.11747327904337, ('R9', 'Move to:R10'): -129.4284600327211, ('R10', 'Move to:R25'): -102.56473048806359, ('mangment', 'Move to:R10'): -130.40405012306434, ('parking', 'Move to:R4'): -121.3499542845349, ('R22', 'Move to:R23'): -88.67221733285801, ('R26', 'Move to:R12'): -138.1665107225143}



In [205]:
q,n,actions =simulate( lambda: Can_Collector.new_instance(HB_basket=4,OSB_basket=0,NB_basket=4,AB_basket=0, init_state= 'Gate1'),n_iterations=1,q=Q, n=n,  f=lambda q, n:q, verbose=False)


   Move to:R12 1
Buskets: 
 Nano:  4 , Helmy:  4 , Academic:  0 , One stop:  0
   Move to:R10 2
Buskets: 
 Nano:  4 , Helmy:  4 , Academic:  0 , One stop:  0
   Move to:R25 3
Buskets: 
 Nano:  4 , Helmy:  4 , Academic:  0 , One stop:  0
   Move to:R23 4
Buskets: 
 Nano:  4 , Helmy:  4 , Academic:  0 , One stop:  0
   Move to:Nano 5
Buskets: 
 Nano:  4 , Helmy:  4 , Academic:  0 , One stop:  0
   Pick 6
Buskets: 
 Nano:  4 , Helmy:  4 , Academic:  0 , One stop:  0
   Move to:R23 7
Buskets: 
 Nano:  3 , Helmy:  4 , Academic:  0 , One stop:  0
   Move to:Nano 8
Buskets: 
 Nano:  3 , Helmy:  4 , Academic:  0 , One stop:  0
   Move to:R23 9
Buskets: 
 Nano:  3 , Helmy:  4 , Academic:  0 , One stop:  0
   Move to:Nano 10
Buskets: 
 Nano:  3 , Helmy:  4 , Academic:  0 , One stop:  0
   Move to:R23 11
Buskets: 
 Nano:  3 , Helmy:  4 , Academic:  0 , One stop:  0
   Move to:Nano 12
Buskets: 
 Nano:  3 , Helmy:  4 , Academic:  0 , One stop:  0
   Move to:R23 13
Buskets: 
 Nano:  3 , Helmy:  4 , 

In [206]:
visualize_path(actions)